# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sqlalchemy as sql
import pandas as pd

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
dbname = 'disaster_response_db'
tbname = 'dr_messages_tbl'

# load data from database
conn = sql.create_engine('sqlite:///{}.db'.format(dbname)).connect()
df = pd.read_sql_table(tbname, con=conn)

In [3]:
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26177.000000,26177.000000,26177.000000,26177.000000,26177.000000,26177.000000,26177.000000,26177.000000,26177.000000,26177.000000,...,26177.000000,26177.000000,26177.000000,26177.000000,26177.000000,26177.000000,26177.000000,26177.000000,26177.000000,26177.000000
mean,15226.345265,0.773580,0.170531,0.004470,0.414104,0.079459,0.050044,0.027658,0.017993,0.032815,...,0.011804,0.043970,0.278336,0.082095,0.093212,0.010773,0.093670,0.020170,0.052565,0.193414
std,8827.039557,0.435345,0.376106,0.066707,0.492576,0.270459,0.218040,0.163994,0.132928,0.178156,...,0.108006,0.205032,0.448188,0.274515,0.290734,0.103234,0.291375,0.140586,0.223168,0.394982
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7449.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15664.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22925.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
#Child alone column only has zeros
print ('Sum of child_alone column: ',df['child_alone'].sum())
df.drop(['child_alone'], axis=1, inplace=True)

Sum of child_alone column:  0


In [5]:
#only the related column has numbers other than 1 and 0, this may be an error, reduce the 2 in this coulmn to a 1
df.groupby('related').count()

,id,message,original,genre,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,,,,,,,,,,,,,,,,,,,,,
0,6115,6115,3390,6115,6115,6115,6115,6115,6115,6115,...,6115,6115,6115,6115,6115,6115,6115,6115,6115,6115
1,19874,19874,6631,19874,19874,19874,19874,19874,19874,19874,...,19874,19874,19874,19874,19874,19874,19874,19874,19874,19874
2,188,188,132,188,188,188,188,188,188,188,...,188,188,188,188,188,188,188,188,188,188


In [6]:
df['related'] = df['related'].apply(lambda x: 1 if x == 2 else x)

In [7]:
X = df.message
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    """
    Input:
    text (str): text from all messages
       
    Returns:
    clean_tokens (list): list of words into numbers of same meaning
    """    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    detected_urls = re.findall(url_regex, text)
    
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")  
        
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)

    text = re.sub(r'[0-9]', " ", text)
    
    tokens = nltk.word_tokenize(text)

    tokens = [word for word in tokens if not word in stopwords.words('english')]

    clean_tokens = []
    
    lemmatizer = nltk.WordNetLemmatizer()
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

In [9]:
#Test tokenize function
tokenize(df.message[100])

['the',
 'message',
 'might',
 'saying',
 'stuck',
 'presidential',
 'palace',
 'pal',
 'since',
 'tuesday',
 'quake',
 'they',
 'need',
 'water',
 'the',
 'message',
 'say',
 'finding',
 'little',
 'water',
 'no',
 'name',
 'number',
 'people',
 'given']

In [10]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        # tokenize by sentences
        sentence_list = nltk.sent_tokenize(text)
        
        for sentence in sentence_list:
            # tokenize each sentence into words and tag part of speech
            pos_tags = nltk.pos_tag(tokenize(sentence))

            # index pos_tags to get the first word and part of speech tag
            
            
            if not len(pos_tags): 
                return False
            first_word, first_tag = pos_tags[0]
            
            # return true if the first word is an appropriate verb or RT for retweet
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True

        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        # apply starting_verb function to all values in X
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
def model_pipe():    
    pipe = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    return pipe

In [12]:
def model_pipeline_rfc():
    pipeline = Pipeline([
                        ('features', FeatureUnion([

                            ('text_pipeline', Pipeline([
                                ('vect', CountVectorizer(tokenizer=tokenize)),
                                 ('tfidf', TfidfTransformer())
                                 ]))
                        ])),
        
                        ('classifier', MultiOutputClassifier(RandomForestClassifier()))
                ])
    return pipeline    

In [13]:
def model_pipeline():
    pipeline = Pipeline([
                        ('features', FeatureUnion([

                            ('text_pipeline', Pipeline([
                                ('vect', CountVectorizer(tokenizer=tokenize)),
                                 ('tfidf', TfidfTransformer())
                                 ]) ),
                            ('starting_verb', StartingVerbExtractor())
                        ])),
                        ('classifier', MultiOutputClassifier(RandomForestClassifier()))
                ])
    return pipeline        

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
model_rfc = model_pipe()
model_rfc.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
y_predicted_test = model_rfc.predict(X_test)
y_prediction_train = model_rfc.predict(X_train)

In [16]:
print (classification_report(y_test, y_predicted_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.85      0.91      0.88      5049
               request       0.76      0.45      0.56      1084
                 offer       0.00      0.00      0.00        30
           aid_related       0.74      0.58      0.65      2705
          medical_help       0.53      0.11      0.18       490
      medical_products       0.64      0.07      0.13       335
     search_and_rescue       0.38      0.04      0.08       187
              security       0.12      0.01      0.02       108
              military       0.62      0.05      0.08       220
                 water       0.85      0.25      0.39       427
                  food       0.82      0.50      0.62       741
               shelter       0.82      0.38      0.52       578
              clothing       0.79      0.17      0.28        88
                 money       0.57      0.05      0.10       152
        missing_people       0.67      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [20]:
#Lets see if the columns tools, hospitals, shops, aid centers and security actually had a value of 1 in the columns
y_test.sum()/len(y_test)

related                   0.771429
request                   0.165623
offer                     0.004584
aid_related               0.413293
medical_help              0.074866
medical_products          0.051184
search_and_rescue         0.028571
security                  0.016501
military                  0.033613
water                     0.065241
food                      0.113216
shelter                   0.088312
clothing                  0.013445
money                     0.023224
missing_people            0.010542
refugees                  0.032391
death                     0.042322
other_aid                 0.131398
infrastructure_related    0.067227
transport                 0.046753
buildings                 0.053323
electricity               0.020474
tools                     0.006723
hospitals                 0.010542
shops                     0.004431
aid_centers               0.011154
other_infrastructure      0.045684
weather_related           0.280519
floods              

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[2, 5, 10]}

cv = GridSearchCV(model_rfc, param_grid=parameters, scoring='f1_micro')
cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.